In [1]:
import warnings
warnings.filterwarnings('ignore') # Filter out warnings
import numpy as np
import pandas as pd
import random as rnd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
from scipy import stats
import math
from math import isnan
import time
from collections import Counter

## Helper function

In [2]:
## SUBJECT TO CHANGE... COLUMNS ARE ADAPTIVE
# Function of getting history for all orders
def GetOrderHistory(df_original):
    df = df_original.copy(deep=False) 
    t = time.time()
    table = {}
    for index, row in df.iterrows():
        if (row['OrderID']!=0):   # don't need records where orderID==0; remove row['Type'], row['Dir'] is okay?
            d = {row['Time_stamp']: [ row['V'],row['P'],row['Label'],row['Level'],\
                                      row['Volts'],row['N_OrderSubmit'],row['bid-ask spread 1'],\
                                      row['Submit_rate'],row['Cancel_rate'],row['Execute_rate'] 
                                    ]}
            if (row['OrderID'] not in table.keys()):
                li = []
                li.append(d)
                table[row['OrderID']] = li
            else:
                table[row['OrderID']].append(d)
    print("Completed. It takes about {0:.2f} minutes.".format((time.time()-t)/60))        
    return table

In [3]:
def ReturnTwoStates_orders(df_original,type_list,orderhistory):
    df = df_original.copy(deep=False)
    t = time.time()
    TwoStates = {}
    # only want records that are of those types, and OrderID != 0
    df = df.loc[df['Type'].isin(type_list)]
    Ids = set(df['OrderID'].tolist())
    Ids.discard(0)
    # look through all order ids in the dictionary
    for k, v in orderhistory.items():
        if k in Ids:
            temp_v = orderhistory[k]
            # avoid cases where there is only one event happened to a single order
            if (len(temp_v)>=2):
                # get first event and the very last event
                st_ed = temp_v[::len(temp_v)-1]
                TwoStates[k] = st_ed
    print("Completed. It takes about {0:.2f} minutes.".format((time.time()-t)/60))  
    return TwoStates

In [50]:
def HistoryToDF_2(history_dict):
    t = time.time()
    colnames = ['OrderID','Time','V','P','Label','Level',\
                'Volts','N_OrderSubmit','Spread','Submit_rate','Cancel_rate','Execute_rate']
    twoStatesName = ['_st','_ed']
    cols = [colnames[0]]
    for i in range(len(twoStatesName)):
        for j in range(1,len(colnames)):
            cols.append(colnames[j]+twoStatesName[i])
    df = pd.DataFrame(columns=cols)
    # traverse through every single orderID
    ct = 0 
    totalItems = len(history_dict.keys())
    steps = int(totalItems/10)
    for k, v in history_dict.items():
        new_row = []
        new_row.append(k)
        new_row.append(list(v[0].keys())[0])
        new_row.extend(list(v[0].values())[0])
        new_row.append(list(v[-1].keys())[0])
        new_row.extend(list(v[-1].values())[0])
        # appending to the dataframe
        df.loc[ct] = new_row
        ct += 1
        if (ct%steps==0):
            print("{0:.2f}%...".format(ct*100/totalItems))
    print("Completed. It takes about {0:.2f} minutes.".format((time.time()-t)/60))
    return df

### Processing starts from here

## READ THE BIG CSV, CONTINUE...

In [5]:
df = pd.read_csv("MsgAndOrders_withNewColumns_noFirst10Mins.csv")
df.head(5)

,Unnamed: 0,Time_stamp,Type,OrderID,V,P,Dir,Time,P_ask_1,V_ask_1,...,mid-price 2,bid-ask spread 3,mid-price 3,bid-ask spread 4,mid-price 4,bid-ask spread 5,mid-price 5,Submit_rate,Cancel_rate,Execute_rate
0,2832,34800.286545,3,28801911,100,2241600,-1,2012-06-21 09:40:00.286545000,2241700,100,...,2239750.0,6100,2239750.0,6300,2239750.0,6600,2239700.0,45.42,38.89,15.70
1,2833,34800.502893,1,28876384,114,2237000,1,2012-06-21 09:40:00.502893000,2241700,100,...,2239850.0,6000,2239800.0,6200,2239800.0,6400,2239800.0,46.03,39.54,14.44
2,2834,34800.602869,3,28850629,100,2242700,-1,2012-06-21 09:40:00.602869000,2241700,100,...,2239900.0,6100,2239850.0,6300,2239850.0,6700,2239950.0,46.12,39.68,14.20
3,2835,34801.121654,3,28578184,100,2241700,-1,2012-06-21 09:40:01.121654000,2242800,100,...,2239950.0,6200,2239900.0,6600,2240000.0,6900,2240050.0,46.07,39.79,14.13
4,2836,34801.121793,1,28889055,114,2237300,1,2012-06-21 09:40:01.121793000,2242800,100,...,2240100.0,6000,2240000.0,6500,2240050.0,6800,2240100.0,46.09,39.78,14.13


In [6]:
print('total column number: ',len(df.columns))
df.columns

total column number:  61


Index(['Unnamed: 0', 'Time_stamp', 'Type', 'OrderID', 'V', 'P', 'Dir', 'Time',
       'P_ask_1', 'V_ask_1', 'P_bid_1', 'V_bid_1', 'P_ask_2', 'V_ask_2',
       'P_bid_2', 'V_bid_2', 'P_ask_3', 'V_ask_3', 'P_bid_3', 'V_bid_3',
       'P_ask_4', 'V_ask_4', 'P_bid_4', 'V_bid_4', 'P_ask_5', 'V_ask_5',
       'P_bid_5', 'V_bid_5', 'Label', 'Level', 'Volts', 'N_OrderSubmit',
       'bid-ask spread 1', 'mid-price 1', 'd_P_ask_51', 'd_P_bid_51',
       'd_P_ask_21', 'd_P_bid_21', 'd_P_ask_32', 'd_P_bid_32', 'd_P_ask_43',
       'd_P_bid_43', 'd_P_ask_54', 'd_P_bid_54', 'Mean_P_ask', 'Mean_P_bid',
       'Mean_V_ask', 'Mean_V_bid', 'P_accu', 'V_accu', 'bid-ask spread 2',
       'mid-price 2', 'bid-ask spread 3', 'mid-price 3', 'bid-ask spread 4',
       'mid-price 4', 'bid-ask spread 5', 'mid-price 5', 'Submit_rate',
       'Cancel_rate', 'Execute_rate'],
      dtype='object')

#### 1) GROUPING ALL MESSAGES BY ORDER IDS

In [7]:
OrderHistory = GetOrderHistory(df)

Completed. It takes about 0.37 minutes.


In [8]:
# take a look at first 1 item of OrderHistory
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a dictionary"
    return dict(islice(iterable, n))
n_items = take(1, OrderHistory.items())
n_items   
# OrderID: [{Time_stamp:[V,P,Label,level,Volts,N_OrderSubmit,bid-ask spread 1,Submit_rate,Cancel_rate,Execute_rate]}]

{28801911: [{34800.286544761: [100,
    2241600,
    'c',
    1.0,
    2550.5179322294466,
    1273,
    4000,
    45.42,
    38.89,
    15.7]}]}

In [10]:
his_list = []
ct_items = 0
for k,v in OrderHistory.items():
    ct_items += 1
    his_list.append(len(v))

print("total number of orders is",int(ct_items))    
print("orders with 2 events is about {0:.2f}%.".format(62518*100/84070))

total number of orders is 84070
orders with 2 events is about 74.36%.


In [11]:
counts_allHistories = Counter(his_list)
counts_allHistories

Counter({1: 19770,
         2: 62518,
         3: 1436,
         4: 205,
         5: 81,
         6: 24,
         7: 12,
         8: 6,
         9: 4,
         10: 3,
         11: 4,
         13: 1,
         14: 1,
         15: 3,
         16: 1,
         18: 1})

In [12]:
ct = 0
for k, v in OrderHistory.items():
    ct += len(v)
print('average number of event for an order is {0:.3f}'.format(ct/len(OrderHistory.keys())))

average number of event for an order is 1.794


#### 2) GETTING START AND END STATES FOR SELECTED ORDER TYPES, STORE IN DICTIONARIES

In [13]:
OE_history = ReturnTwoStates_orders(df_original=df,type_list=[4,5],orderhistory=OrderHistory)

Completed. It takes about 0.00 minutes.


In [14]:
OC_history = ReturnTwoStates_orders(df_original=df,type_list=[2,3],orderhistory=OrderHistory)

Completed. It takes about 0.00 minutes.


In [17]:
print("total number of execution orders and cancellation orders are {}, {} respectively."
      .format(len(OE_history.keys()),len(OC_history.keys())))

total number of execution orders and cancellation orders are 6293, 58459 respectively.


#### 3) FORMAT ORDER HISTORIES INTO DATAFRAMES

In [18]:
df.columns

Index(['Unnamed: 0', 'Time_stamp', 'Type', 'OrderID', 'V', 'P', 'Dir', 'Time',
       'P_ask_1', 'V_ask_1', 'P_bid_1', 'V_bid_1', 'P_ask_2', 'V_ask_2',
       'P_bid_2', 'V_bid_2', 'P_ask_3', 'V_ask_3', 'P_bid_3', 'V_bid_3',
       'P_ask_4', 'V_ask_4', 'P_bid_4', 'V_bid_4', 'P_ask_5', 'V_ask_5',
       'P_bid_5', 'V_bid_5', 'Label', 'Level', 'Volts', 'N_OrderSubmit',
       'bid-ask spread 1', 'mid-price 1', 'd_P_ask_51', 'd_P_bid_51',
       'd_P_ask_21', 'd_P_bid_21', 'd_P_ask_32', 'd_P_bid_32', 'd_P_ask_43',
       'd_P_bid_43', 'd_P_ask_54', 'd_P_bid_54', 'Mean_P_ask', 'Mean_P_bid',
       'Mean_V_ask', 'Mean_V_bid', 'P_accu', 'V_accu', 'bid-ask spread 2',
       'mid-price 2', 'bid-ask spread 3', 'mid-price 3', 'bid-ask spread 4',
       'mid-price 4', 'bid-ask spread 5', 'mid-price 5', 'Submit_rate',
       'Cancel_rate', 'Execute_rate'],
      dtype='object')

In [24]:
n_items

{28801911: [{34800.286544761: [100,
    2241600,
    'c',
    1.0,
    2550.5179322294466,
    1273,
    4000,
    45.42,
    38.89,
    15.7]}]}

In [51]:
OE_df = HistoryToDF_2(OE_history)

10.00%...
19.99%...
29.99%...
39.98%...
49.98%...
59.97%...
69.97%...
79.96%...
89.96%...
99.95%...
Completed. It takes about 0.89 minutes.


In [ ]:
OC_df = HistoryToDF_2(OC_history)

10.00%...
20.00%...
30.00%...
39.99%...
49.99%...
59.99%...


## TESTING.... 

In [47]:
OE_df[['Level_st','Level_ed']].head(5)

,OrderID,Time_st,V_st,P_st,Label_st,Level_st,Volts_st,N_OrderSubmit_st,Spread_st,Submit_rate_st,...,V_ed,P_ed,Label_ed,Level_ed,Volts_ed,N_OrderSubmit_ed,Spread_ed,Submit_rate_ed,Cancel_rate_ed,Execute_rate_ed
0,29035435,34811.182036,100,2241400,la,1.0,2748.633074,1186,3700,46.29,...,99,2241400,mb,1.0,2782.883809,1181,1800,46.64,40.24,13.11
1,29092003,34814.305412,75,2240100,la,1.0,2784.540430,1183,2000,46.43,...,39,2240100,mb,1.0,2791.681226,1176,3700,46.61,40.07,13.32
2,29107250,34814.477257,2,2240000,la,1.0,2784.540430,1187,3300,46.46,...,2,2240000,mb,1.0,2786.894273,1179,3400,46.44,39.86,13.71
3,29171582,34816.945843,11,2240000,lb,1.0,2786.894273,1179,100,46.44,...,11,2240000,ma,1.0,2783.408919,1179,3400,46.42,39.84,13.74
4,29189328,34818.023053,90,2240600,la,2.0,2788.729411,1180,3400,46.46,...,90,2240600,mb,1.0,2775.738294,1182,1800,46.63,39.96,13.41
